In [1]:
import sys
import os
sys.path.append('../../')

%load_ext autoreload
%autoreload 2
%matplotlib inline

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from skimage.io import imread

from gen.load_data import load_data

In [3]:
from sklearn.utils import shuffle

train_df, valid_df, test_df = load_data('../../data')


print(train_df.head())


                                               image                   id  \
0  ../../data/Train/CameraRGB/episode_0002_000287...  episode_0002_000287   
1  ../../data/Train/CameraRGB/episode_0008_000112...  episode_0008_000112   
2                 ../../data/Train/CameraRGB/804.png                  804   
3  ../../data/Train/CameraRGB/episode_0008_000286...  episode_0008_000286   
4               ../../data/Train/CameraRGB/M2-25.png                M2-25   

                                               label  
0  ../../data/Train/CameraSeg/episode_0002_000287...  
1  ../../data/Train/CameraSeg/episode_0008_000112...  
2                 ../../data/Train/CameraSeg/804.png  
3  ../../data/Train/CameraSeg/episode_0008_000286...  
4               ../../data/Train/CameraSeg/M2-25.png  


In [4]:
from models.segnet import model_segnetVGG16, model_segnet

model = model_segnet(3, image_shape=(416, 800, 3))
model.summary()

/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Build enceder done..
Build decoder done..
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 416, 800, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 416, 800, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 416, 800, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 416, 800, 64) 0           batch_normalization_1[0][0]      
___________________________________________________________________

In [11]:
from gen.datagen import oversample_generator_from_df, balanced_generator_from_df

BATCH_SIZE = 2
model_dir = '../../saved_models/segnet/segnet_v4/'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

train_gen = oversample_generator_from_df(train_df, BATCH_SIZE, (416, 800))
valid_gen = balanced_generator_from_df(valid_df, BATCH_SIZE, (416, 800))

In [12]:
gpus = 1

# model_gpu = multi_gpu_model(model, gpus)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
from train import train_nn

m = train_df.shape[0]
history = train_nn(model, 
                   train_gen, 
                   valid_gen, 
                   training_size=4000, 
                   batch_size=BATCH_SIZE,
                   validation_size=valid_df.shape[0],
                   output_path=model_dir, 
                   epochs=100,
                   gpus = 1)

Epoch 1/100
2000/2000 [==============================] - 1743s 872ms/step - loss: 0.1650 - acc: 0.9456 - val_loss: 0.2653 - val_acc: 0.9243

Epoch 00001: val_loss improved from inf to 0.26532, saving model to ../../saved_models/segnet/segnet_v4//model.hdf5
Epoch 2/100
 897/2000 [============>.................] - ETA: 11:21 - loss: 0.0909 - acc: 0.9704

In [ ]:
# # summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# model.load_weights('../../saved_models/segnet/segnet_v3//model.hdf5')
model.reset_states()
model_gpu.load_weights('../../saved_models/segnet/segnet_v4//model.hdf5')
model.save('../../saved_models/segnet/segnet_v4/model_saved.h5')